Adapted from https://github.com/davidzheng814/GoBot/blob/master/create_training_data.py
and https://github.com/davesque/go.py/blob/master/go/board.py

# Chargement du ficher sfg et création des états de jeu

In [67]:
import os
import numpy as np
from multiprocessing import Process
import sys

In [62]:
# créer la liste contenant la liste des coups joués dans le sgf
def sgf_to_moves_list(filename):
    f = open(filename, 'r')
    content = f.read()
    f.close()

    if 'AB[' in content or 'AW[' in content:
        return False

    moves = []
    ind = 0

    while True:
        if len(moves) % 2 == 0:
            index = content.find(';B[', ind)
            if index == -1:
                break
        else:
            index = content.find(';W[', ind)
            if index == -1:
                break

        if content[index + 3] == ']':
            moves.append('pass')
            ind = index + 4
            continue

        x, y = [ord(letter) - ord('a') for letter in content[index+3:index+5]]

        moves.append((x, y))
        ind = index + 6

    return moves

In [74]:
moves=sgf_to_moves_list("Mi_Yuting-Park_Junghwan.sgf")
moves

[(15, 3),
 (3, 15),
 (16, 15),
 (3, 2),
 (13, 16),
 (16, 9),
 (2, 4),
 (3, 7),
 (16, 7),
 (2, 3),
 (4, 3),
 (3, 3),
 (2, 7),
 (2, 6),
 (3, 6),
 (2, 8),
 (1, 7),
 (4, 6),
 (3, 5),
 (4, 5),
 (1, 8),
 (3, 8),
 (2, 9),
 (9, 3),
 (3, 9),
 (4, 4),
 (11, 2),
 (13, 2),
 (9, 2),
 (8, 2),
 (8, 3),
 (10, 2),
 (9, 1),
 (10, 3),
 (10, 1),
 (11, 3),
 (7, 2),
 (8, 4),
 (8, 1),
 (16, 5),
 (15, 5),
 (15, 4),
 (16, 4),
 (14, 4),
 (17, 5),
 (16, 3),
 (17, 4),
 (16, 2),
 (12, 2),
 (2, 13),
 (1, 5),
 (16, 12),
 (13, 1),
 (14, 2),
 (3, 17),
 (5, 17),
 (2, 16),
 (2, 15),
 (1, 17),
 (1, 10),
 (4, 16),
 (1, 9),
 (2, 11),
 (1, 11),
 (5, 9),
 (5, 8),
 (1, 3),
 (6, 9),
 (5, 10),
 (6, 8),
 (5, 14),
 (4, 13),
 (6, 12),
 (8, 15),
 (6, 16),
 (5, 12),
 (2, 12),
 (2, 10),
 (5, 13),
 (6, 11),
 (1, 12),
 (1, 13),
 (8, 14),
 (7, 12),
 (0, 13),
 (0, 14),
 (0, 12),
 (3, 11),
 (3, 12),
 (4, 12),
 (3, 10),
 (4, 11),
 (1, 14),
 (4, 9),
 (9, 15),
 (7, 15),
 (4, 10),
 (6, 10),
 (7, 14),
 (6, 15),
 (4, 14),
 (3, 13),
 (5, 16),
 (

In [17]:
# paramètre initiaux
taille = 19
in_bounds = lambda x, y: 0 <= x < taille and 0 <= y < taille


In [20]:
#init
board = [[-1 for _ in range(19)] for __ in range(19)]
group_map = [[-1 for _ in range(19)] for __ in range(19)]
groups = []
illegal = [[False for _ in range(19)] for __ in range(19)]
empty_groups = []

num_nonpass_moves = len(moves) - moves.count('pass')
inputs = [[None for _ in range(8)] for __ in range(num_nonpass_moves)]
targets = [None for _ in range(num_nonpass_moves)]
ind = 0

In [21]:
board

[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1

In [13]:
def update_board(board, group_map, groups, empty_groups, illegal, move, player):
    '''
        board: board[y][x] == 0 if (x, y) is black, 1 if (x, y) is white, -1 if empty
        group_map: group_map[y][x] == group_index of (x, y), -1 if (x, y) is empty
        groups: list of groups indexed by group_index
                Each group contains (liberties, pieces)
                liberites is a set of all (x, y) liberties of the group
                pieces is a set of all stones in the group
                elements can be None if group has been removed
        empty_groups: list of indices of all empty groups
        illegal: illegal[y][x] == 1 if move is illegal, 0 otherwise
        move: (x, y) of move
        player: 0 if black is moving, 1 if white is moving
    '''

    x, y = move
    opponent = 1 - player
    board[y][x] = player

    liberties = set()
    allies = set()
    enemies = set()
    for x2, y2 in [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]:
        if in_bounds(x2, y2):
            if board[y2][x2] == -1:
                liberties.add((x2, y2))
            elif board[y2][x2] == player:
                allies.add(group_map[y2][x2])
            else:
                enemies.add(group_map[y2][x2])

    if len(allies) == 0:
        new_group = (liberties, set([move]))
        if empty_groups:
            group_ind = empty_groups.pop()
            groups[group_ind] = new_group
        else:
            groups.append(new_group)
            group_ind = len(groups) - 1

        group_map[y][x] = group_ind

    else:
        group_ind = min(allies)
        group_map[y][x] = group_ind

        new_liberties, new_pieces = groups[group_ind]
        for other_index in allies:
            if other_index == group_ind:
                continue
            other_liberties, other_pieces = groups[other_index]
            new_liberties.update(other_liberties)
            new_pieces.update(other_pieces)
            temp = groups[other_index]
            for x2, y2 in other_pieces:
                group_map[y2][x2] = group_ind

            groups[other_index] = None
            empty_groups.append(other_index)

        new_liberties.remove((x, y))

        new_pieces.add((x, y))
        new_liberties.update(liberties)

    captured = []
    for enemy in enemies:
        groups[enemy][0].remove((x, y))
        if len(groups[enemy][0]) == 0:
            pieces = groups[enemy][1]

            captured.extend(pieces)
            for x2, y2 in pieces:
                board[y2][x2] = -1
                group_map[y2][x2] = -1

                for x3, y3 in [(x2-1, y2), (x2+1, y2), (x2, y2-1), (x2, y2+1)]:
                    if in_bounds(x3, y3) and board[y3][x3] == player:
                        groups[group_map[y3][x3]][0].add((x2, y2))

            groups[enemy] = None
            empty_groups.append(enemy)


In [28]:
    for i, move in enumerate(moves):
        player = i % 2

        if move == 'pass':
            illegal = [[False for _ in range(19)] for __ in range(19)]
            fill_illegal(board, group_map, groups, illegal, player)
            continue

#        add_board_to_array(board, group_map, groups, illegal, player, inputs, ind)

#        add_move_to_array(move, targets, ind)
        illegal = [[False for _ in range(19)] for __ in range(19)]
        update_board(board, group_map, groups, empty_groups, illegal, move, player)
        
        ind += 1


In [29]:
board

[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, -1, -1, 0, -1, -1, -1, -1, -1],
 [-1, 1, -1, 1, -1, -1, -1, 0, -1, 0, 1, 0, 0, 1, 1, -1, 1, 1, -1],
 [1, 0, 1, 1, 0, -1, -1, -1, 0, 1, 1, 1, 0, -1, 1, 0, 1, 0, 1],
 [0, -1, 0, -1, 1, -1, -1, -1, 1, -1, -1, -1, -1, 0, 1, 1, 0, 0, -1],
 [-1, 0, 0, 0, 1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 1, 0, -1, 0, -1],
 [0, -1, -1, 0, 1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 0, -1, -1],
 [-1, 0, 0, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, 0, -1, -1],
 [1, 0, 1, 1, 0, 1, 1, -1, -1, -1, -1, -1, -1, 1, 0, -1, 0, -1, -1],
 [-1, 1, 0, 0, -1, 0, 1, -1, -1, -1, -1, -1, 1, -1, 1, 1, 1, -1, -1],
 [-1, 1, 1, 0, 0, 0, 1, -1, -1, -1, -1, -1, -1, 1, 0, 1, -1, 1, -1],
 [-1, 1, 0, 1, 1, -1, 1, -1, 0, -1, 1, 1, 1, -1, 0, 1, -1, 0, -1],
 [0, 0, 0, 0, 1, 1, -1, 1, 0, 1, -1, 0, 0, 0, -1, 0, 1, -1, -1],
 [0, 1, 1, 1, 1, 0, 1, 0, 1, 1, -1, -1, -1, -1, -1, 0, 1, -1, -1],
 [0, 0, -1, -1, 0, 0,

In [31]:
def print_small_board(board, illegal):
    for i in range(taille):
        text = ""
        for j in range(taille):
            text += ('o' if illegal[i][j] else '-') if not (0 <= board[i][j] <= 1) else ('B' if board[i][j] == 0 else 'W')
        print text


In [32]:
print_small_board(board, illegal)

-------------------
--------BBB--B-----
-W-W---B-BWBBWW-WW-
WBWWB---BWWWB-WBWBW
B-B-W---W----BWWBB-
-BBBW--------BWB-B-
B--BW--------WWWB--
-BBWW---------B-B--
WBWWBWW------WB-B--
-WBB-BW-----W-WWW--
-WWBBBW------WBW-W-
-WBWW-W-B-WWW-BW-B-
BBBBWW-WBW-BBB-BW--
BWWWWBWBWW-----BW--
BB--BBBBBWW-W-BWBW-
B-WW--WWWBB--B-WB-B
--B-BBB--BWWWB-WB--
-B-B-W---BWBBBW-WB-
---------------WBB-


In [48]:
def sgf_to_matrice(filename,numero_de_coup):
    moves = sgf_to_moves_list(filename)
    if not moves:
        return False
    #initialise le plateau
    board = [[-1 for _ in range(19)] for __ in range(19)]
    group_map = [[-1 for _ in range(19)] for __ in range(19)]
    groups = []
    illegal = [[False for _ in range(19)] for __ in range(19)]
    empty_groups = []

    num_nonpass_moves = len(moves) - moves.count('pass')
    inputs = [[None for _ in range(8)] for __ in range(num_nonpass_moves)]
    targets = [None for _ in range(num_nonpass_moves)]
    ind = 0
    
    #joue jusqu'au coup voulu
    for i, move in enumerate(moves):
        if i<=numero_de_coup:
            player = i % 2

            if move == 'pass':
                illegal = [[False for _ in range(19)] for __ in range(19)]
                fill_illegal(board, group_map, groups, illegal, player)
                continue

#        add_board_to_array(board, group_map, groups, illegal, player, inputs, ind)

#        add_move_to_array(move, targets, ind)
            illegal = [[False for _ in range(19)] for __ in range(19)]
            update_board(board, group_map, groups, empty_groups, illegal, move, player)
        
            ind += 1

    return (board,illegal)



In [69]:
i=0
while i<2:
    (board,illegal) = sgf_to_matrice("Mi_Yuting-Park_Junghwan.sgf",i)
    print_small_board(board, illegal) 
    print "coup %s" %i
    i=i+1

-------------------
-------------------
-------------------
---------------B---
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
coup 0
-------------------
-------------------
-------------------
---------------B---
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
---W---------------
-------------------
-------------------
-------------------
coup 1


# Enregistrer les états du jeu sous le format de "davesque"

In [103]:
from go import Board, BoardError, View, clear, getch

In [111]:
board2=Board(19)
view = View(board2)

In [110]:
print view

X . . . . . . .
. . . . . . . .
. . + . . + . .
. . . . . . . .
. . . . . . . .
. . + . . + . .
. . . . . . . .
. . . . . . . .


In [186]:
def move2(taille,moves):
    self=Board(taille)
    liste_des_etats_de_jeu=[]
    i=0
    while i< len (moves):
        liste_des_etats_de_jeu.append(self)
        i+=1

    for i, move in enumerate(moves):

        """
        Makes a move at the given location for the current turn's color.
        """
        x=move[0]+1
        y=move[1]+1
        # Check if coordinates are occupied
        if self[x, y] is not self.EMPTY:
            raise BoardError('Cannot move on top of another piece!')

        # Store history and make move
        self._push_history()
        self[x, y] = self._turn

        # Check if any pieces have been taken
        taken = self._take_pieces(x, y)

        # Check if move is suicidal.  A suicidal move is a move that takes no
        # pieces and is played on a coordinate which has no liberties.
        if taken == 0:
            self._check_for_suicide(x, y)

        # Check if move is redundant.  A redundant move is one that would
        # return the board to the state at the time of a player's last move.
        self._check_for_ko()

        self._flip_turn()
        self._redo = []
        
        liste_des_etats_de_jeu[i]=self
    return liste_des_etats_de_jeu

In [187]:
liste_des_etats_de_jeu=move2(19,moves)
i =0
while i<10:
    view = View(liste_des_etats_de_jeu[i])
    print view
    i=i+1

X . . . . . . . . . . . . . . . . . .
. . . . . . . . * * * . . * . . . . .
. o . o . . . * . * o * * o o . o o .
o * o o * . . . * o o o * . o * o * o
* . * . o . . . o . . . . * o o * * .
. * * * o . . . . . . . . * o * . * .
* . . * o . . . . . . . . o o o * . .
. * * o o . . . . . . . . . * . * . .
o * o o * o o . . . . . . o * . * . .
. o * * . * o . . + . . o . o o o . .
. o o * * * o . . . . . . o * o . o .
. o * o o . o . * . o o o . * o . * .
* * * * o o . o * o . * * * . * o . .
* o o o o * o * o o . . . . . * o . .
* * . . * * * * * o o . o . * o * o .
* . o o . . o o o * * . . * . o * . *
. . * . * * * . . * o o o * . o * . .
. * . * . o . . . * o * * * o . o * .
. . . . . . . . . . . . . . . o * * .
X . . . . . . . . . . . . . . . . . .
. . . . . . . . * * * . . * . . . . .
. o . o . . . * . * o * * o o . o o .
o * o o * . . . * o o o * . o * o * o
* . * . o . . . o . . . . * o o * * .
. * * * o . . . . . . . . * o * . * .
* . . * o . . . . . . . . o o o * . .
. * * o o . 

In [178]:
board2

In [142]:
print board2.TURNS

(Black, White)


In [143]:
board3=Board(19)
print board3.turn


Black


In [152]:
board3._state

State(board=[[Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty], [Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty], [Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty], [Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty], [Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty], [Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty], [Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty, Empty], [Empty, Empty, Empty, Empty, Empty, Empty,

In [164]:
from copy import copy
